In [1]:
import torch
import numpy as np
import sklearn
import os
from architecture import PointNetSegmentation

print('Loading data... ', end='')
data_dir = '../dataset/S3DIS'
f = np.load(os.path.join(data_dir, 'data_test.npz'))
data, labels = f['data'], f['labels']
in_features = data.shape[1]
batch_size=32

if not torch.cuda.is_available():
    data = data[:2*batch_size, ...]
    labels = labels[:2*batch_size, ...]

# zero-center xyz and rgb
data[:, :6, :] -= np.mean(data[:, :6, :], axis=2, keepdims=True)
# zero-center location
data[:, 6:, :] = data[:, 6:, :] * 2.0 - 1.0

X = torch.from_numpy(data.astype(np.float32))
t = torch.from_numpy(labels.astype(np.int64))

print('Done')

if torch.cuda.is_available():
    pn_segment = PointNetSegmentation(in_features=in_features, num_classes=13).cuda()
    pn_segment.load_state_dict(torch.load('../results/segmentation/93.40/PointNetSegment.pt'))
else:
    pn_segment = PointNetSegmentation(in_features=in_features, num_classes=13)
    pn_segment.load_state_dict(torch.load('../results/segmentation/93.40/PointNetSegment.pt', map_location=lambda storage, location: storage))


pn_segment.train(False)
num_batches = X.shape[0] // batch_size
corr = []
y = []
for bn in range(num_batches):
    X_batch = torch.autograd.Variable(X[bn * batch_size: bn * batch_size + batch_size, ...])
    t_batch = t[bn * batch_size: bn * batch_size + batch_size, ...]
    if torch.cuda.is_available():
        X_batch = X_batch.cuda()
        t_batch = t_batch.cuda()
    y_batch = pn_segment(X_batch)[0].data.max(1)[1]
    corr.append((y_batch == t_batch).cpu().numpy().sum(1) / X.shape[2])
    y.append(y_batch.cpu().numpy())

X_test = np.transpose(X.numpy(), (0, 2, 1)) # num_images x num_points x 9
y_test = np.concatenate(y, axis=0) # num_images x num_points
t_test = t[:y_test.shape[0]].numpy() # num_images x num_points
acc = np.concatenate(corr) # num_images
print('Test Accuracy: {:.4f}'.format(acc.sum() / X.shape[0]))

tp = 0
p = 0
for i in range(13):
    tp += np.logical_and(t_test == i, y_test == i).sum()
    p += (y_test == i).sum()
print('Test Precision: {:.4f}'.format(tp / p))

ImportError: No module named 'architecture'

In [28]:
import pandas as pd
from pyntcloud import PyntCloud
classes = ['ceiling', 'floor', 'wall', 'beam', 'column', 'window', 'door', 'table', 'chair', 'sofa', 'bookcase', 'board', 'clutter']
class2color = {'ceiling':     [0,255,0],
               'floor':       [0,0,255],
               'wall':        [0,255,255],
               'beam':        [255,255,0],
               'column':      [255,0,255],
               'window':      [100,100,255],
               'door':        [200,200,100],
               'table':       [170,120,200],
               'chair':       [255,0,0],
               'sofa':        [200,100,100],
               'bookcase':    [10,200,100],
               'board':       [200,200,200],
               'clutter':     [50,50,50]}
label2color = [class2color[c] for c in classes]

In [50]:
i = 2
print('Sample {} Accuracy: {:.4f}'.format(i, acc[i]))
positions = X_test[i, :, :3] # num_points * 3
points = pd.DataFrame(positions, columns=['x', 'y', 'z'])

colors = list(map(lambda x : label2color[x], y_test[i])) # num_points * 3 (uint8)
points[['red', 'blue', 'green']] = pd.DataFrame(colors, index=points.index)

cloud = PyntCloud(points)
cloud.plot(point_size=0.03)

Sample 2 Accuracy: 1.0000


In [36]:
colors = list(map(lambda x : label2color[x], t_test[i]))
points[['red', 'blue', 'green']] = pd.DataFrame(colors, index=points.index)

cloud = PyntCloud(points)
cloud.plot(point_size=0.03)